In [20]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms,datasets

transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)),transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
train=datasets.ImageFolder('./PetImages/train',transform)
test=datasets.ImageFolder('./PetImages/test',transform)
trainDataLoader=DataLoader(train,batch_size=64,shuffle=True)
testDataLoader=DataLoader(test,batch_size=64,shuffle=False)

import torch.nn as nn # AlexNet
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #step1
        self.Conv_Pool=nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5,stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3,stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        #step2
        self.fc=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p=0.5),
            nn.Linear(4096, 2),
        )
    def forward(self,x):
        x=self.Conv_Pool(x)
        
        x=x.view(-1,256*6*6)
        x=self.fc(x)
        return x

learningRate=0.0001



In [2]:
net=CNN()#訓練
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learningRate)




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net=net.to(device)
for epoch in range(10):
    for i,(input,label) in enumerate(trainDataLoader,0):#0是下標起始位置預設為0
        # data 的格式[inputs, labels]   
        #初始為0，清除上個batch的梯度訊息
        optimizer.zero_grad()         
        #前向+後向+優化
        outputs = net(input.to(device))
        loss = criterion(outputs,label.to(device))
        loss.backward()
        optimizer.step()

    
        if i%100 == 0:
            
            print('[%d,%5d] loss :%.3f' %
                 (epoch+1,i+1,loss.item()))
            running_loss = 0.0

PATH = './CatVsDog.pth'
torch.save(net.state_dict(), PATH)#保存訓練結果

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[1,    1] loss :0.694


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


[1,  101] loss :0.693
[1,  201] loss :0.693
[1,  301] loss :0.633
[2,    1] loss :0.683
[2,  101] loss :0.587
[2,  201] loss :0.666
[2,  301] loss :0.555
[3,    1] loss :0.508


KeyboardInterrupt: 

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_net=CNN()#利用train/test data測試訓練成果 
PATH = './alexnet0.9.pth'
test_net=test_net.to(device)
test_net.load_state_dict(torch.load(PATH))

total=0
correct=0

with torch.no_grad():
    for i,data in enumerate(trainDataLoader,0):
        input,label=data
        test_out=test_net(input.to(device))
        _,predicted=torch.max(test_out.data,dim=1)
        total+=label.size(0)
        correct+=(predicted==label.to(device)).sum().item()
    print('Accuracy of the network on the  train images: ',float(100*correct/total),'%')

    total=0
    correct=0
    for i,data in enumerate(testDataLoader,0):
        input,label=data
        test_out=test_net(input.to(device))
        _,predicted=torch.max(test_out.data,dim=1)
        total+=label.size(0)
        correct+=(predicted==label.to(device)).sum().item()
    print('Accuracy of the network on the  test images: ',float(100*correct/total),'%')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Accuracy of the network on the  train images:  98.50185461727443 %
Accuracy of the network on the  test images:  90.4047619047619 %


In [15]:
import torch 
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
import os
from PIL import Image     #利用網上圖片測試訓練成果

transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)),transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
for jpg in os.listdir('./test_with_internet_photo'):
    img=transform(Image.open('./test_with_internet_photo/{}'.format(jpg)))



    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    internet_test_net=CNN()
    PATH = './CatVsDog0.9.pth'
    internet_test_net=internet_test_net.to(device)
    internet_test_net.load_state_dict(torch.load(PATH))

    with torch.no_grad():
        output=internet_test_net(img.to(device))
        _,predicted=torch.max(output.data,dim=1)
        if predicted.tolist()==[0]:#0=cat,1=dog
            print('I guess {} is a cat.\n'.format(jpg)) 
        if predicted.tolist()==[1]:#0=cat,1=dog
            print('I guess {} is a dog.\n'.format(jpg)) 



I guess 0.jpg is a cat.

I guess 1.jpg is a cat.

I guess 10.jpg is a cat.

I guess 2.jpg is a cat.

I guess 3.jpg is a cat.

I guess 4.jpg is a cat.

I guess 5.jpg is a dog.

I guess 6.jpg is a dog.

I guess 7.jpg is a dog.

I guess 8.jpg is a dog.

I guess 9.jpg is a dog.

